In [ ]:
import numpy as np
import cv2

def visualize(input, faces,color, thickness=2):
    if faces[1] is not None:
        for idx, face in enumerate(faces[1]):
            coords = face[:-1].astype(np.int32)
            cv2.rectangle(input, (coords[0], coords[1]), (coords[0]+coords[2], coords[1]+coords[3]), color, thickness)


detector = cv2.FaceDetectorYN.create(
    "model/face_detection_yunet_2022mar.onnx",
    "",
    (320, 320),
    0.8,
    0.3,
    5000
)

image1 = cv2.imread("path/to/image")
img1 = image1.copy()
img1Width = int(img1.shape[1])
img1Height = int(img1.shape[0])
img1 = cv2.resize(img1, (img1Width, img1Height))

detector.setInputSize((img1Width, img1Height))
faces1 = detector.detect(img1)


# Create the face recognition model
recognizer = cv2.FaceRecognizerSF.create(
    "model/face_recognition_sface_2021dec.onnx", "")
face1_align = recognizer.alignCrop(image1, faces1[1][0])
face1_feature = recognizer.feature(face1_align)
cap = cv2.VideoCapture(0)
while True:
    ret, image2 = cap.read()
    img2 = image2.copy()

    detector.setInputSize((img2.shape[1], img2.shape[0]))
    faces2 = detector.detect(img2)

    if recognizer is not None and faces2[1] is not None and len(faces2[1]) > 0:
        face2_align = recognizer.alignCrop(image2, faces2[1][0])
                # Extract features
        face2_feature = recognizer.feature(face2_align)
        l2_similarity_threshold = 1.128

        l2_score = recognizer.match(face1_feature, face2_feature, cv2.FaceRecognizerSF_FR_NORM_L2)

        if l2_score <= l2_similarity_threshold:
            visualize(img2,faces2,(0, 255, 0))
        else:
            visualize(img2,faces2,(0, 0, 255))

    cv2.imshow('image', img2)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()
